## Introduzione alla Regressione - Lezione 1

#### Mettiamolo in prospettiva

✅ Esistono molti tipi di metodi di regressione, e quale scegliere dipende dalla risposta che stai cercando. Se vuoi prevedere l'altezza probabile di una persona di una certa età, useresti la `regressione lineare`, poiché stai cercando un **valore numerico**. Se invece sei interessato a scoprire se un tipo di cucina dovrebbe essere considerato vegano o meno, stai cercando un **assegnamento di categoria**, quindi utilizzeresti la `regressione logistica`. Imparerai di più sulla regressione logistica più avanti. Rifletti un po' su alcune domande che puoi porre ai dati e su quale di questi metodi sarebbe più appropriato.

In questa sezione, lavorerai con un [piccolo dataset sul diabete](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Immagina di voler testare un trattamento per pazienti diabetici. I modelli di Machine Learning potrebbero aiutarti a determinare quali pazienti risponderebbero meglio al trattamento, basandoti su combinazioni di variabili. Anche un modello di regressione molto semplice, se visualizzato, potrebbe mostrare informazioni sulle variabili che ti aiuterebbero a organizzare i tuoi studi clinici teorici.

Detto ciò, iniziamo con questo compito!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>

<!--![Illustrazione di \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.it.jpg)<br>Illustrazione di @allison_horst-->


## 1. Caricare il nostro set di strumenti

Per questo compito, avremo bisogno dei seguenti pacchetti:

-   `tidyverse`: Il [tidyverse](https://www.tidyverse.org/) è una [collezione di pacchetti R](https://www.tidyverse.org/packages) progettata per rendere la scienza dei dati più veloce, facile e divertente!

-   `tidymodels`: Il framework [tidymodels](https://www.tidymodels.org/) è una [collezione di pacchetti](https://www.tidymodels.org/packages/) per la modellazione e il machine learning.

Puoi installarli con il seguente comando:

`install.packages(c("tidyverse", "tidymodels"))`

Lo script qui sotto verifica se hai i pacchetti necessari per completare questo modulo e li installa per te nel caso in cui alcuni siano mancanti.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Ora, carichiamo questi fantastici pacchetti e rendiamoli disponibili nella nostra sessione R corrente. (Questo è solo a scopo illustrativo, `pacman::p_load()` lo ha già fatto per te)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Il dataset sul diabete

In questo esercizio, metteremo in pratica le nostre competenze di regressione facendo previsioni su un dataset relativo al diabete. Il [dataset sul diabete](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) include `442 campioni` di dati sul diabete, con 10 variabili predittive: `età`, `sesso`, `indice di massa corporea`, `pressione sanguigna media` e `sei misurazioni del siero ematico`, oltre a una variabile di risultato `y`: una misura quantitativa della progressione della malattia un anno dopo la baseline.

|Numero di osservazioni|442|
|----------------------|:---|
|Numero di predittori|Le prime 10 colonne sono predittive numeriche|
|Risultato/Target|La colonna 11 è una misura quantitativa della progressione della malattia un anno dopo la baseline|
|Informazioni sui predittori|- età in anni
||- sesso
||- bmi indice di massa corporea
||- bp pressione sanguigna media
||- s1 tc, colesterolo totale nel siero
||- s2 ldl, lipoproteine a bassa densità
||- s3 hdl, lipoproteine ad alta densità
||- s4 tch, colesterolo totale / HDL
||- s5 ltg, probabilmente logaritmo del livello di trigliceridi nel siero
||- s6 glu, livello di zucchero nel sangue|

> 🎓 Ricorda, questo è apprendimento supervisionato, e abbiamo bisogno di un target denominato 'y'.

Prima di poter manipolare i dati con R, è necessario importare i dati nella memoria di R o stabilire una connessione ai dati che R possa utilizzare per accedervi in remoto.

> Il pacchetto [readr](https://readr.tidyverse.org/), che fa parte del Tidyverse, offre un modo rapido e intuitivo per leggere dati rettangolari in R.

Ora, carichiamo il dataset sul diabete fornito in questo URL sorgente: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Inoltre, eseguiremo un controllo di base sui nostri dati utilizzando `glimpse()` e mostreremo le prime 5 righe con `slice()`.

Prima di procedere, introduciamo anche qualcosa che incontrerai spesso nel codice R 🥁🥁: l'operatore pipe `%>%`

L'operatore pipe (`%>%`) esegue operazioni in sequenza logica passando un oggetto a una funzione o a un'espressione di chiamata. Puoi pensare all'operatore pipe come a dire "e poi" nel tuo codice.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` ci mostra che questi dati contengono 442 righe e 11 colonne, con tutte le colonne di tipo `double`.

<br>

> glimpse() e slice() sono funzioni di [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, parte del Tidyverse, è una grammatica per la manipolazione dei dati che offre un insieme coerente di verbi per affrontare le sfide più comuni nella manipolazione dei dati.

<br>

Ora che abbiamo i dati, concentriamoci su una caratteristica (`bmi`) come obiettivo per questo esercizio. Questo richiederà di selezionare le colonne desiderate. Quindi, come possiamo farlo?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) ci permette di *selezionare* (e opzionalmente rinominare) colonne in un data frame.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Dati di addestramento e test

È una pratica comune nell'apprendimento supervisionato *suddividere* i dati in due sottoinsiemi: un set (tipicamente più grande) con cui addestrare il modello e un set più piccolo "di riserva" con cui verificare come si è comportato il modello.

Ora che abbiamo i dati pronti, possiamo vedere se una macchina può aiutare a determinare una suddivisione logica tra i numeri in questo dataset. Possiamo utilizzare il pacchetto [rsample](https://tidymodels.github.io/rsample/), che fa parte del framework Tidymodels, per creare un oggetto che contiene le informazioni su *come* suddividere i dati, e poi altre due funzioni di rsample per estrarre i set di addestramento e test creati:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Addestrare un modello di regressione lineare con Tidymodels

Ora siamo pronti per addestrare il nostro modello!

In Tidymodels, i modelli vengono specificati utilizzando `parsnip()` definendo tre concetti:

-   Il **tipo** di modello distingue tra modelli come la regressione lineare, la regressione logistica, i modelli ad albero decisionale e così via.

-   La **modalità** del modello include opzioni comuni come regressione e classificazione; alcuni tipi di modelli supportano entrambe le modalità, mentre altri ne hanno solo una.

-   Il **motore** del modello è lo strumento computazionale che verrà utilizzato per adattare il modello. Spesso si tratta di pacchetti R, come **`"lm"`** o **`"ranger"`**.

Queste informazioni sul modello vengono catturate in una specifica del modello, quindi creiamone una!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Dopo che un modello è stato *specificato*, il modello può essere `stimato` o `addestrato` utilizzando la funzione [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), tipicamente usando una formula e alcuni dati.

`y ~ .` significa che adatteremo `y` come quantità/obiettivo previsto, spiegato da tutti i predittori/caratteristiche, cioè `.` (in questo caso, abbiamo solo un predittore: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Dai risultati del modello, possiamo osservare i coefficienti appresi durante l'addestramento. Essi rappresentano i coefficienti della retta di miglior adattamento che ci fornisce l'errore complessivo più basso tra la variabile reale e quella prevista.
<br>

## 5. Fare previsioni sul set di test

Ora che abbiamo addestrato un modello, possiamo utilizzarlo per prevedere la progressione della malattia y per il dataset di test usando [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Questo sarà utilizzato per tracciare la linea tra i gruppi di dati.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Evviva! 💃🕺 Abbiamo appena addestrato un modello e lo abbiamo utilizzato per fare previsioni!

Quando si effettuano previsioni, la convenzione di tidymodels è di produrre sempre un tibble/data frame di risultati con nomi di colonne standardizzati. Questo rende facile combinare i dati originali e le previsioni in un formato utilizzabile per operazioni successive, come la creazione di grafici.

`dplyr::bind_cols()` unisce in modo efficiente più data frame per colonna.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Tracciare i risultati del modello

Ora è il momento di vedere tutto questo visivamente 📈. Creeremo un grafico a dispersione di tutti i valori `y` e `bmi` del set di test, quindi utilizzeremo le previsioni per tracciare una linea nella posizione più appropriata, tra i raggruppamenti di dati del modello.

R ha diversi sistemi per creare grafici, ma `ggplot2` è uno dei più eleganti e versatili. Questo ti permette di comporre grafici **combinando componenti indipendenti**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Pensa un attimo a cosa sta succedendo qui. Una linea retta attraversa molti piccoli punti di dati, ma cosa sta facendo esattamente? Riesci a capire come potresti usare questa linea per prevedere dove un nuovo punto dati, mai visto prima, dovrebbe posizionarsi in relazione all'asse y del grafico? Prova a mettere in parole l'utilità pratica di questo modello.

Congratulazioni, hai costruito il tuo primo modello di regressione lineare, hai creato una previsione con esso e l'hai mostrata in un grafico!



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
